<a href="https://colab.research.google.com/github/amittkatz/Information-Retrieval-/blob/main/Information_Retrieval_using_Boolean_Query_Almog_Edri%26Amit_Katz.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

first,importing necessary libraries

In [1]:
import requests #for making HTTP requests in Python
from bs4 import BeautifulSoup # pulling data from HTML or XML files

define the query - we chose to check the symptoms of alzheimer

In [2]:
query = "Alzheimer symptoms"

In [3]:
r = requests.get('https://www.google.com/search?q={}'.format(query))

In [4]:
soup = BeautifulSoup(r.text, "html.parser")

In HTML, the<  a href > attribute provides us with link for the web page. To obtain all such links, we run the following lines of 

In [5]:
links = []
for item in soup.find_all('a'):
    links.append(item.get('href'))

The links to third-party webpages,, start with the following characters, “/url?q=”, so we need to filter out such strings.

In [6]:
final = []
for item in links:
    s = item[0:7]
    if s == "/url?q=":
        final.append(item)

after we have the links we need, we will save all the data of the 10 first links in a list

In [7]:
# save as list
All_links_list = []
for Link in range(1,11):
  webpage1 = requests.get('https://www.google.com/' + final[Link]) #final[0] refers to the first web page link
  webpagetext = BeautifulSoup(webpage1.text, "html.parser")
  all_p = webpagetext.find_all('p')
  text = ""
  for item in all_p:
    text = text + item.get_text()
  All_links_list.append(text.lower())

simplfying the information

In [8]:
import string
import nltk
nltk.download('stopwords')
nltk.download('punkt')
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer, PorterStemmer
from nltk.tokenize import sent_tokenize , word_tokenize
import glob
import re
import os
import numpy as np
import sys
Stopwords = set(stopwords.words('english'))

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


def functions to clean our data from stopwords,punctuation and etc

In [9]:
def Punc (text):
  text_p = "".join([char for char in text if char not in string.punctuation])
  return text_p

def remove_special_characters(text):
    regex = re.compile('[^a-zA-Z0-9\s]')
    text_returned = re.sub(regex,'',text)
    return text_returned


def Token (text):
  words = word_tokenize(text)
  return words 

def FilterList (text):
  filtered_words = [word for word in text if word not in Stopwords]
  return filtered_words     


from nltk.stem.porter import PorterStemmer
porter = PorterStemmer()

def Stem (text):
  stemmed = [porter.stem(word) for word in text]
  return stemmed

apply the funcs on our list of docs

In [10]:

no_punc_list = []
for t in All_links_list:
 t_no_punc = Punc(t)
 t_no_punc = remove_special_characters(t)
 no_punc_list.append(t_no_punc)



words_list = []
for w in no_punc_list:
  words_ = Token(w)
  words_list.append(words_)

filtered_words_list = []
for f in words_list:
  f_list = FilterList(f)
  filtered_words_list.append(f_list)  


stemmed_list = []
for s in filtered_words_list:
  s_list = Stem(s)
  stemmed_list.append(s_list)

create a df that contains the words and docs

In [11]:
from numpy.core import numeric
# Create data frame
import pandas as pd

num=1
for text in stemmed_list:
  doc = "doc" + str(num)
  text.insert(0,doc)
  num+=1

df = pd.DataFrame(stemmed_list)

In [12]:
# Export to csv file
df.to_csv('words.csv', encoding = 'utf-8-sig', index = False)

We created a csv file of all document where each document in a different row.

In [13]:
import pandas
# module to read the contents of the file from a csv file

from contextlib import redirect_stdout
# module to redirect the output to a text file

terms = []
# list to store the terms present in the documents

keys = []
# list to store the names of the documents

vec_Dic = {}
# dictionary to store the name of the document and the boolean vector as list

dicti = {}
# dictionary to store the name of the document and the terms present in it as a
# vector

dummy_List = []
# list for performing some operations and clearing them


def filter(documents, rows, cols):
	'''function to read and separate the name of the documents and the terms
	present in it to a separate list from the data frame and also create a
	dictionary which has the name of the document as key and the terms present in
	it as the list of strings which is the value of the key'''

	for i in range(rows):
		for j in range(cols):
			# traversal through the data frame

			if(j == 0):
				# first column has the name of the document in the csv file
				keys.append(documents.loc[i].iat[j])
			else:
				dummy_List.append(documents.loc[i].iat[j])
				# dummy list to update the terms in the dictionary

				if documents.loc[i].iat[j] not in terms:
					# add the terms to the list if it is not present else continue
					terms.append(documents.loc[i].iat[j])

		copy = dummy_List.copy()
		# copying the dummy list to a different list

		dicti.update({documents.loc[i].iat[0]: copy})
		# adding the key value pair to a dictionary

		dummy_List.clear()
		# clearing the dummy list


def bool_Representation(dicti, rows, cols):
	'''In this function we get a boolean representation of the terms present in the
	documents in the form of lists, later we create a dictionary which contains
	the name of the documents as key and value as the list of boolean values
	representing the terms present in the document'''

	# terms.sort()
	# we sort the elements in the alphabetical order for the convience, the order
	# of the term does not make any difference

	for i in (dicti):
		# for every document in the dictionary we check for each string present in
		# the list

		for j in terms:
			# if the string is present in the list we append 1 else we append 0

			if j in dicti[i]:
				dummy_List.append(1)
			else:
				dummy_List.append(0)
			# appending 1 or 0 for obtaining the boolean representation

		copy = dummy_List.copy()
		# copying the dummy list to a different list

		vec_Dic.update({i: copy})
		# adding the key value pair to a dictionary

		dummy_List.clear()
		# clearing the dummy list

def query_Vector(query):
	'''In this function we represent the query in the form of boolean vector'''

	qvect = []
	# query vector which is returned at the end of the function

	for i in terms:
		# if the word present in the list of terms is also present in the query
		# then append 1 else append 0

		if i in query:
			qvect.append(1)
 
		else:
			qvect.append(0)

	return qvect
	# return the query vector which is obtained in the boolean form


def prediction(q_Vect):
	'''In this function we make the prediction regarding which document is related
	to the given query by performing the boolean operations'''

	dictionary = {}
	listi = []
	count = 0
	# initialisation of the dictionary , list and a variable which is further
	# required for performing the computation

	term_Len = len(terms)
	# number of terms present in the term list

	for i in vec_Dic:
		# for every document in the dictionary containing the terms present in it
		# the form of boolean vector

		for t in range(term_Len):
			if(q_Vect[t] == vec_Dic[i][t]):
				# if the words present in the query is also present in the
				# document or if the words present in the query is also absent in
				# the document

				count += 1
				# increase the value of count variable by one
				# the condition in which words present in document and absent in
				#query , present in query and absent in document is not considered

		dictionary.update({i: count})
		# dictionary updation here the name of the document is the key and the
		# count variable computed earlier is the value

		count = 0
		# reinitialisaion of count variable to 0

	for i in dictionary:
		listi.append(dictionary[i])
		# here we append the count value to list

	listi = sorted(listi, reverse=True)
	# we sort the list in the descending order which is needed to rank the
	#documents according to the relevance

	ans = ' '
	# variable to store the name of the document which is most relevant
	

	with open('output.txt', 'w') as f:
		with redirect_stdout(f):
			# to redirect the output to a text file

			print("ranking of the documents")

			for count, i in enumerate(listi):
				key = check(dictionary, i)
				# Function call to get the key when the value is known
				if count == 0:
					ans = key
					# to store the name of the document which is most relevant

				print(key, "rank is", count+1)
				# print the name of the document along with its rank

				dictionary.pop(key)
				# remove the key from the dictionary after printing

			print(ans, "is the most relevant document for the given query")
			# to print the name of the document which is most relevant


def check(dictionary, val):
	'''Function to return the key when the value is known'''

	for key, value in dictionary.items():
		if(val == value):
			# if the given value is same as the value present in the dictionary
			# return the key

			return key


In [14]:
documents = pandas.read_csv(r'words.csv')
# to read the data from the csv file as a dataframe

rows = len(documents)
# to get the number of rows

cols = len(documents.columns)
# to get the number of columns

In [15]:
display(documents)

,0,1,2,3,4,5,6,7,8,9,...,1115,1116,1117,1118,1119,1120,1121,1122,1123,1124
0,doc1,call,24,hour,seven,day,week,helplin,8002723900let,us,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,doc2,call,24,hour,seven,day,week,helplin,8002723900let,us,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,doc3,call,24,hour,seven,day,week,helplin,8002723900let,us,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,doc4,call,24,hour,seven,day,week,helplin,8002723900let,us,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,doc5,call,24,hour,seven,day,week,helplin,8002723900let,us,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,doc6,call,24,hour,seven,day,week,helplin,8002723900let,us,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,doc7,featur,conditionsalzheim,diseas,progress,neurolog,disord,caus,brain,shrink,...,inform,verifi,site,compli,honcod,standard,trustworthi,health,inform,verifi
7,doc8,offici,websit,unit,state,governmenther,know,gov,mean,offici,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,doc9,back,alzheim,diseas,symptom,alzheim,diseas,progress,slowli,sever,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,doc10,whether,your,concern,someon,care,import,know,warn,sign,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [16]:
filter(documents, rows, cols)
# function call to read and separate the name of the documents and the terms
# present in it to a separate list from the data frame and also create a
# dictionary which has the name of the document as key and the terms present in
# it as the list of strings which is the value of the key

bool_Representation(dicti, rows, cols)
# In this function we get a boolean representation of the terms present in the
# documents in the form of lists, later we create a dictionary which contains
# the name of the documents as key and value as the list of boolean values
#representing the terms present in the document



In [17]:
bool_matrix = np.array([vec_Dic[i] for i in list(vec_Dic.keys())])
df = pd.DataFrame(bool_matrix, index=list(vec_Dic.keys()))
dfb_matrix = np.array([vec_Dic[i] for i in list(vec_Dic.keys())])
df = pd.DataFrame(bool_matrix, index=list(vec_Dic.keys()))
df

,0,1,2,3,4,5,6,7,8,9,...,1259,1260,1261,1262,1263,1264,1265,1266,1267,1268
doc1,1,1,1,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,0,0,0
doc2,1,1,1,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,0,0,0
doc3,1,1,1,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,0,0,0
doc4,1,1,1,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,0,0,0
doc5,1,1,1,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,0,0,0
doc6,1,1,1,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,0,0,0
doc7,0,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
doc8,1,0,0,0,0,0,0,0,1,1,...,0,0,0,0,0,0,0,0,0,0
doc9,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
doc10,0,0,0,0,1,1,0,0,1,0,...,1,1,1,1,1,1,1,1,1,0


In [18]:
from typing import Dict
print("Enter query")
query = input()
# to get the query input from the user, the below input is given for obtaining
# the output as in output.txt file
query = word_tokenize(query)

query = [word for word in query if len(query)>1 ]
query = [word for word in query if word not in Stopwords]

q_Vect = query_Vector(query)
# function call to represent the query in the form of boolean vector

print(q_Vect)
num = []
for i in q_Vect:
  if i == 1:
    num.append(i)
print("number of docs with the query:")    
print(len(num))

prediction(q_Vect)
# Function call to make the prediction regarding which document is related to
# the given query by performing the boolean operations
# k = []
# term_Len = len(terms)
# for i in vec_Dic:
# 		for t in range(term_Len):
# 			if(q_Vect[t] == vec_Dic[i][t]):
# 				for l in vec_Dic.keys():
#           if (l == i):
#             k.append(key)
          
            
# print(k)


Enter query
risk and factor
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

part 2

creating an inverred index to the doc - now we can see the location of each word ( in what docs, the position of the word and how many time it was reapited).

In [19]:
inverted_index = {}
for key, value in dicti.items():
    for indx, word in enumerate(value):
        inverted_index.setdefault(word, {}).setdefault(key, []).append(indx)

for value in inverted_index.values():
    for list_ in value.values():
        list_[:] = [len(list_), list(list_)]

print(inverted_index)

{'call': {'doc1': [2, [0, 233]], 'doc2': [1, [0]], 'doc3': [1, [0]], 'doc4': [1, [0]], 'doc5': [1, [0]], 'doc6': [3, [0, 439, 588]], 'doc8': [5, [112, 117, 142, 258, 430]]}, '24': {'doc1': [1, [1]], 'doc2': [1, [1]], 'doc3': [1, [1]], 'doc4': [1, [1]], 'doc5': [1, [1]], 'doc6': [1, [1]]}, 'hour': {'doc1': [1, [2]], 'doc2': [1, [2]], 'doc3': [1, [2]], 'doc4': [1, [2]], 'doc5': [1, [2]], 'doc6': [1, [2]]}, 'seven': {'doc1': [1, [3]], 'doc2': [1, [3]], 'doc3': [1, [3]], 'doc4': [1, [3]], 'doc5': [1, [3]], 'doc6': [1, [3]]}, 'day': {'doc1': [3, [4, 174, 477]], 'doc2': [1, [4]], 'doc3': [1, [4]], 'doc4': [1, [4]], 'doc5': [1, [4]], 'doc6': [1, [4]], 'doc10': [1, [112]]}, 'week': {'doc1': [2, [5, 175]], 'doc2': [1, [5]], 'doc3': [1, [5]], 'doc4': [1, [5]], 'doc5': [1, [5]], 'doc6': [1, [5]], 'doc10': [1, [113]]}, 'helplin': {'doc1': [2, [6, 148]], 'doc2': [1, [6]], 'doc3': [1, [6]], 'doc4': [1, [6]], 'doc5': [1, [6]], 'doc6': [2, [6, 441]]}, '8002723900let': {'doc1': [1, [7]], 'doc2': [1, [7

Risk and factor query

In [20]:
query = "risk and factor"
query = query.split()

list_of_risk = []
list_of_factor = []
i = query[0] 
ii = query[2]
if i in inverted_index:
  for key in inverted_index[i].keys():
    list_of_risk.append(key)
if ii in inverted_index:
    for key in inverted_index[ii].keys():
     list_of_factor.append(key) 


list_of_risk_as_set = set(list_of_risk)
intersection = list_of_risk_as_set.intersection(list_of_factor)


intersection_as_list = list(intersection)

print("The docs with risk and factor are:")
print(intersection_as_list)


The docs with risk and factor are:
['doc6', 'doc7']


Precision,recall and f-score calculation

In [21]:
num_all_docs_retreived = len(links)
num_rlv_docs = len(final)
num_rlv_docs_retreived = len(intersection_as_list)

def precision(num_rlv_docs_retreived,num_all_docs_retreived):
  precision = num_rlv_docs_retreived/num_all_docs_retreived
  return precision


def recall(num_rlv_docs_retreived,num_rlv_docs):
  recall = num_rlv_docs_retreived/num_rlv_docs
  return recall

def fscore(precision,recall):
  fscore = 2*precision*recall/precision*recall
  return fscore

print("precision:")
print(precision(num_rlv_docs_retreived,num_all_docs_retreived))

print("recall:")
print(recall(num_rlv_docs_retreived,num_rlv_docs))

print("fscore:")
fscore(precision(num_rlv_docs_retreived,num_all_docs_retreived),recall(num_rlv_docs_retreived,num_rlv_docs))

precision:
0.03389830508474576
recall:
0.11764705882352941
fscore:


0.027681660899653977

In [22]:
len(final)

17

runing the code with 25 links insteat of 10

In [23]:
# save as list
links_list25 = []
for Link in range(len(final)):
  webpage1 = requests.get('https://www.google.com/' + final[Link]) #final[0] refers to the first web page link
  webpagetext = BeautifulSoup(webpage1.text, "html.parser")
  all_p = webpagetext.find_all('p')
  text = ""
  for item in all_p:
    text = text + item.get_text()
  links_list25.append(text.lower())

In [24]:
no_punc_list = []
for t in links_list25:
 t_no_punc = Punc(t)
 t_no_punc = remove_special_characters(t)
 no_punc_list.append(t_no_punc)

words_list = []
for w in no_punc_list:
  words_ = Token(w)
  words_list.append(words_)

filtered_words_list = []
for f in words_list:
  f_list = FilterList(f)
  filtered_words_list.append(f_list)  


stemmed_list25 = []
for s in filtered_words_list:
  s_list = Stem(s)
  stemmed_list25.append(s_list)

In [25]:
num=1
for text in stemmed_list25:
  doc = "doc" + str(num)
  text.insert(0,doc)
  num+=1

df = pd.DataFrame(stemmed_list25)

In [26]:
# Export to csv file
df.to_csv('words25.csv', encoding = 'utf-8-sig', index = False)

In [27]:
documents = pandas.read_csv(r'words25.csv')
# to read the data from the csv file as a dataframe

rows = len(documents)
# to get the number of rows

cols = len(documents.columns)
# to get the number of columns

In [28]:
display(documents)

,0,1,2,3,4,5,6,7,8,9,...,1271,1272,1273,1274,1275,1276,1277,1278,1279,1280
0,doc1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,doc2,call,24,hour,seven,day,week,helplin,8002723900let,us,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,doc3,call,24,hour,seven,day,week,helplin,8002723900let,us,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,doc4,call,24,hour,seven,day,week,helplin,8002723900let,us,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,doc5,call,24,hour,seven,day,week,helplin,8002723900let,us,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,doc6,call,24,hour,seven,day,week,helplin,8002723900let,us,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,doc7,call,24,hour,seven,day,week,helplin,8002723900let,us,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,doc8,featur,conditionsalzheim,diseas,progress,neurolog,disord,caus,brain,shrink,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,doc9,offici,websit,unit,state,governmenther,know,gov,mean,offici,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,doc10,back,alzheim,diseas,symptom,alzheim,diseas,progress,slowli,sever,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [29]:
filter(documents, rows, cols)
# function call to read and separate the name of the documents and the terms
# present in it to a separate list from the data frame and also create a
# dictionary which has the name of the document as key and the terms present in
# it as the list of strings which is the value of the key

bool_Representation(dicti, rows, cols)
# In this function we get a boolean representation of the terms present in the
# documents in the form of lists, later we create a dictionary which contains
# the name of the documents as key and value as the list of boolean values
#representing the terms present in the document

In [30]:
inverted_index = {}
for key, value in dicti.items():
    for indx, word in enumerate(value):
        inverted_index.setdefault(word, {}).setdefault(key, []).append(indx)

for value in inverted_index.values():
    for list_ in value.values():
        list_[:] = [len(list_), list(list_)]

print(inverted_index)

{nan: {'doc1': [1277, [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199, 200, 201, 202, 203, 204, 205, 206, 207, 208, 209, 210, 211, 212, 213, 214, 215, 216, 21

In [31]:
query = "risk and factor"
query = query.split()

list_of_risk = []
list_of_factor = []
i = query[0] 
ii = query[2]
if i in inverted_index:
  for key in inverted_index[i].keys():
    list_of_risk.append(key)
if ii in inverted_index:
    for key in inverted_index[ii].keys():
     list_of_factor.append(key) 


list_of_risk_as_set = set(list_of_risk)
intersection = list_of_risk_as_set.intersection(list_of_factor)


intersection_as_list = list(intersection)

print("The docs with risk and factor are:")
print(intersection_as_list)


The docs with risk and factor are:
['doc13', 'doc12', 'doc8', 'doc7']


In [32]:
num_all_docs_retreived = len(links)
num_rlv_docs = len(final)
num_rlv_docs_retreived = len(intersection_as_list)

def precision(num_rlv_docs_retreived,num_all_docs_retreived):
  precision = num_rlv_docs_retreived/num_all_docs_retreived
  return precision


def recall(num_rlv_docs_retreived,num_rlv_docs):
  recall = num_rlv_docs_retreived/num_rlv_docs
  return recall

def fscore(precision,recall):
  fscore = 2*precision*recall/precision*recall
  return fscore

print("precision:")
print(precision(num_rlv_docs_retreived,num_all_docs_retreived))

print("recall:")
print(recall(num_rlv_docs_retreived,num_rlv_docs))

print("fscore:")
fscore(precision(num_rlv_docs_retreived,num_all_docs_retreived),recall(num_rlv_docs_retreived,num_rlv_docs))

precision:
0.06779661016949153
recall:
0.23529411764705882
fscore:


0.1107266435986159

In [33]:
bool_matrix = np.array([vec_Dic[i] for i in list(vec_Dic.keys())])
df = pd.DataFrame(bool_matrix, index=list(vec_Dic.keys()))
dfb_matrix = np.array([vec_Dic[i] for i in list(vec_Dic.keys())])
df = pd.DataFrame(bool_matrix, index=list(vec_Dic.keys()))
df

,0,1,2,3,4,5,6,7,8,9,...,1803,1804,1805,1806,1807,1808,1809,1810,1811,1812
doc1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
doc2,1,1,1,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,0,0,0
doc3,1,1,1,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,0,0,0
doc4,1,1,1,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,0,0,0
doc5,1,1,1,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,0,0,0
doc6,1,1,1,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,0,0,0
doc7,1,1,1,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,0,0,0
doc8,0,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
doc9,1,0,0,0,0,0,0,0,1,1,...,0,0,0,0,0,0,0,0,0,0
doc10,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [34]:
from typing import Dict
print("Enter query")
query = input()
# to get the query input from the user, the below input is given for obtaining
# the output as in output.txt file
query = word_tokenize(query)

query = [word for word in query if len(query)>1 ]
query = [word for word in query if word not in Stopwords]

q_Vect = query_Vector(query)
# function call to represent the query in the form of boolean vector

print(q_Vect)
num = []
for i in q_Vect:
  if i == 1:
    num.append(i)
print("number of docs with the query:")    
print(len(num))


prediction(q_Vect)
# Function call to make the prediction regarding which document is related to
# the given query by performing the boolean operations



Enter query
risk and factor
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,